### Probability Sharpe Ratio

In [1]:
import numpy as np

class ProbSharpe :
    def __init__(
            self, series, seed, delta, maxIter, bounds = None
    ):
        # Construct the object
        self.series, self.w, self.delta = series, seed, delta
        self.z, self.d1Z=None, [None for i in range(series.shape[1])]
        self.maxIter, self.iter, self.obs = maxIter,0,series.shape[0]
        if len(bounds) == None or seed.shape[0]!=len(bounds):
            self.bounds = [(0,1) for i in seed]
        else :
            self.bounds = bounds
    
    def optimize(self) :
        # Optimize weights
        mean = [self.get_moments(self.series[:,i],1) for i in range(self.series.shape[1])]
        w = np.array(self.w)
        # derivatives
        while True :
            if self.iter == self.maxIter : break
            # compute gradient
            d1Z, z = self.get_d1Zs(mean, w)
            # evaluate result
            if z > self.z and self.check_bounds(w) == True : 
                # Store new local optimum
                self.z, self.d1Z = z, d1Z
                self.w = np.array(w)
            #Find direction and normalize
            self.iter += 1
            w = self.step_size(w, d1Z)
            if w == None : return 
    
    def check_bounds(self, w) :
        # Check that boundary conditions are satisfied
        flag = True
        for i in range(w.shape[0]) :
            if w[i, 0] < self.bounds[i][0] : flag = False
            if w[i, 0] > self.bounds[i][1] : flag = False
        return flag
    
    def step_size(self, w, d1Z) :
        # determine step size for next iteration
        x = dict()
        for i in range(len(d1Z)) :
            if d1Z[i] != 0 : x[abs(d1Z[i])] = 1
        if len(x) == 0 : return
        index = x[max(x)]
        w[index, 0] += self.delta / d1Z[index]
        w /= sum(w)
        return w
    
    def get_d1Zs(self, mean, w) :
        # first order derivatives of Z
        d1Z = [0 for i in range(self.series.shape[1])]
        m = [0 for i in range(4)]
        series = np.dot(self.series, w)[:, 0]
        m[0] = self.get_moments(series, 1)
        for i in range(1, 4) :
            m[i] = self.get_moments(series, i + 1, m[0])
        stats = self.get_stats(m)
        
        meanSR, sigmaSR = self.get_sharpe_ratio(stats, self.obs)
        
        for i in range(self.series.shape[1]) :
            d1Z[1] = self.get_d1Z(stats, m, meanSR, sigmaSR, mean, w, i)
        
        return d1Z, meanSR / sigmaSR
    
    def get_d1Z(self, stats, m, meanSR, sigmaSR, mean, w, index):
        # First order derivatives of Z with respect to index
        d1Mu = self.get_mu(mean, index)
        d1Sigma = self.get_sigma(stats[1], mean, w, index)
        d1Skew = self.get_skew(d1Sigma, stats[1], mean, w, index, m[2])
        d1Kurt = self.get_kurt(d1Sigma,stats[1],mean,w,index,m[3])
        
        d1meanSR = (d1Mu * stats[1] - d1Sigma * stats[0]) / stats[1] ** 2
        d1sigmaSR = (d1Kurt * meanSR ** 2 + 2 * meanSR * d1meanSR * (stats[3] - 1)) / 4
        d1sigmaSR -= d1Skew * meanSR + d1meanSR * stats[2]
        d1sigmaSR /= 2 * sigmaSR * (self.obs - 1)
        d1Z = (d1meanSR * sigmaSR - d1sigmaSR * meanSR)/sigmaSR ** 2
        return d1Z
    
    def get_mu(self, mean, index) :
        # first order derivatives of Mu
        return mean[index]

SyntaxError: incomplete input (4018852810.py, line 21)